In [1]:
import pandas as pd
import papermill as pm
import scrapbook as sb
import os

D:\Anaconda3\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
def last_two_complete_weeks(ts):
    if pd.Timestamp(ts).hour < 23:
        last_complete_day = pd.Timestamp(ts) - pd.Timedelta(days=1)
    else:
        last_complete_day = pd.Timestamp(ts)   
    day_of_week = last_complete_day.dayofweek
    days_delta = day_of_week if day_of_week < 6 else -1
    end_date = last_complete_day - pd.Timedelta(days=days_delta)
    start_date = end_date - pd.Timedelta(days=14)   
    
    return str(start_date.date()), str(end_date.date())

# last_two_complete_weeks('2021-11-14 22:00:00')  ('2021-10-25', '2021-11-08')
# last_two_complete_weeks('2021-11-14 23:00:00')  ('2021-11-01', '2021-11-15')
# last_two_complete_weeks('2021-11-15 22:00:00')  ('2021-11-01', '2021-11-15')
# last_two_complete_weeks('2021-11-15 23:00:00')  ('2021-11-01', '2021-11-15')

In [3]:
tokens = pd.read_sql('''
SELECT 
    DISTINCT token
FROM 
    ohlc
''', 'sqlite:///data/data.db').values.ravel().tolist()  # ['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV', 'AAVE', 'COMP']

In [4]:
ts_max = pd.read_sql('''
SELECT 
    MAX(ts)
FROM 
    ohlc
''', 'sqlite:///data/data.db').values.ravel().tolist()[0]  # '2021-12-15 00:00:00'

start_date, end_date = last_two_complete_weeks(ts_max)  # '2021-11-29', '2021-12-13'

In [5]:
base_output_folder = os.path.expanduser('output')

if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

In [6]:
for token in tokens:
    print(f'running for {token}', end='\r')
    res = pm.execute_notebook(
        'template.ipynb',
        f'{base_output_folder}/market_analysis_{token}_between_{start_date}_and_{end_date}(excluded).ipynb',
        parameters = {
            'start_date': start_date,
            'end_date': end_date,
            'token': token})

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

Executing:   0%|          | 0/11 [00:00<?, ?cell/s]

In [7]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'avgReturn': scrap['df'].data['return'].mean(),
        'avgVolatility': scrap['df'].data['volatility'].mean(),
        'maximumDrawdown': scrap['df'].data['maximumDrawdown'].min(),
        'rangeOfClose': scrap['df'].data['close'].iloc[-1] - scrap['df'].data['close'].iloc[0]
    }) for book, scrap in sb.read_notebooks('output').notebook_scraps.items()])

,token,avgReturn,avgVolatility,maximumDrawdown,rangeOfClose
0,AAVE,-7.697463e-04,0.212356,-0.378718,-58.6450
1,ADA,-4.241618e-04,0.217477,-0.312827,-0.2445
2,ATOM,-1.776982e-04,0.299425,-0.329570,-2.7560
3,AVAX,-5.540979e-04,0.286536,-0.385790,-22.9300
4,BTC,-3.569235e-04,0.154894,-0.206462,-6999.8400
5,COMP,-1.015116e-03,0.224354,-0.377871,-88.2800
6,CRV,-5.385247e-04,0.328177,-0.394862,-1.0362
7,DOT,-4.540131e-04,0.245633,-0.350800,-6.0050
8,ETH,-6.065377e-05,0.174104,-0.188101,-141.7100
9,SOL,-3.446384e-04,0.242432,-0.317207,-26.6870
